In [78]:
from sklearn.cluster import KMeans

from sklearn.linear_model import LinearRegression

from sklearn.ensemble import IsolationForest

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PolynomialFeatures

from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import mutual_info_regression
from sklearn.neighbors import LocalOutlierFactor

from category_encoders import OrdinalEncoder
from category_encoders.one_hot import OneHotEncoder

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

### Helpers

In [2]:
def make_clusters(df, col, y, n_cluster, merge=False):
    '''
        function to boil down a colum to n_cluster
        
        recive:
        
            df: a data frame 
            col: col to make cluster
            y: the response variable
            n_cluster: stamitation of number of cluster
            
        return:  
            a data frame with 'col' droped, and 'col' + '_cluster' column
            is added to the data frame

        
    '''
    col_clusters = KMeans(n_clusters=n_cluster, random_state=777)

    # 'Neighborhood' and 'MSSubClass' stats
    col_stats = df.groupby(col)[y].describe()

    # Getting clusters
    col_clusters.fit(col_stats)

    # preparing DF with cluster lables to merge
    new_name = col + '_Cluster'
    col_cluster_df = pd.DataFrame( { col: col_stats.index.to_list(),
                                     new_name: col_clusters.labels_.tolist()} )
    
    if merge:
        # merging the clusters with the data frame
        df = df.merge(col_cluster_df, how='left', on=col)

        df[new_name] = df[new_name].astype(str)
        
        result = df.drop(columns=col)
        
    else:
        result = col_cluster_df
    
    return result.copy()


# Low frequency categories to 'Other'
def lower_than(df, col, percentage):
    ''' function that will merge low frequency classes into 
        a single class 'Others'

        parameters:
            df: a DataFrame
            col: column's name to work on
            percentage(%): the threshold like 0.1, 0.2

        returns:
            df: the data frame with col's classes that are lowers
                than 'threshold' been repleced with 'Other' category
    '''

    # calculating the column frequency
    col_freq = df[col].value_counts(normalize=True)

    # the getting the column threshold 
    threshold = col_freq.quantile(q= percentage)

    # knowing the classes that are below the threshold
    less_freq_classes = col_freq[ col_freq <= threshold ]

    others = less_freq_classes.index.to_list()

    print(others)

    df[col] = df[col].replace(others, 'Others')

    return df

# Sort correlation matrix without duplicate
def corr_pair(df, cols, threshold = 0.7):
    '''
    function that calculate the corrolation of features
    and filtering just unique pairs 
    '''

    corr = df[cols].corr().abs()

    # creating a matrix same size matrix as 'tr'
    # [1 1 1]
    # [1 1 1]
    # [1 1 1]
    temp_matrix = np.ones(corr.shape)

    # selecting the upper triagle of the matrix
    # excluting the diagonal
    # [0 1 1]
    # [0 0 1]
    # [0 0 0]
    upper_tri = np.triu(temp_matrix, k=1)

    # converting to booleans
    # [FALSE TRUE TRUE]
    # [FALSE FLASE TRUE]
    # [FALSE FALSE FALSE]
    uppper_tri_bool = upper_tri.astype( bool )

    # filterin the corroletion matrix
    upper_corr = corr.where( uppper_tri_bool )

    unique_corr_pair = upper_corr.unstack().dropna()
    sorted_pairs = unique_corr_pair.sort_values(ascending=False)

    return sorted_pairs[ sorted_pairs > threshold ].copy()

### Data

In [3]:
def data(verbose=False):
    
    d_tr = pd.read_csv('/Users/antirrabia/Documents/01-GitHub/DataMining-_-/CSV/HousePrices/train.csv', index_col='Id')
    d_te = pd.read_csv('/Users/antirrabia/Documents/01-GitHub/DataMining-_-/CSV/HousePrices/test.csv', index_col='Id')
    
    # Utilities has just 2 categories, and one of them
    # just appears once so we delete the whole column.
    d_tr = d_tr.drop(columns='Utilities')
    d_te = d_te.drop(columns='Utilities')
    
    # Marking Training Set
    d_tr['Training'] = True
    d_te['Training'] = False  
    
    if(verbose):
        print('d_tr shape:', d_tr.shape)
        print('d_te shape:', d_te.shape)
        
    return (d_tr.copy(), d_te.copy())
    

### Analazy 'Y'('SalePrice')

In [4]:
def analyze_y(df, verbose=False):
    
    
    df = df.copy()
    
    # Plotting
    if(verbose):
        fg, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2, figsize=(11,7))
        # chequing the distribution of 'y' = 'SalePrice'
        
        sns.histplot( df['SalePrice'], bins=50, ax= ax1);
        ax1.title.set_text('Skewed SalePrice')
    
        sns.scatterplot(x=df['GrLivArea'], y=df['SalePrice'], ax=ax3);
        ax3.title.set_text('Outliers')
    
    outliers = df[ df['GrLivArea'] >= 4500].index
    
    # deleting the outliers in 'GrLivArea'
    df.drop(outliers, inplace=True)
    
    # Plotting
    if(verbose):
        # cheking again
        sns.scatterplot(x=df['GrLivArea'], y=df['SalePrice'], ax=ax4);
        ax4.title.set_text('NO Outliers')
    
    # pipeline to scale and do a powerTransforme on y('SalePrice')
    fix_y = Pipeline([('scaler', RobustScaler()), ('power', PowerTransformer(method='yeo-johnson'))])
    
    y = fix_y.fit_transform( df['SalePrice'].values.reshape(-1, 1) )
    
    # Plotting
    if(verbose):
        # checking the distribution of 'y' again
        sns.histplot(y, bins=50, ax=ax2);
        ax2.title.set_text('yeo-johnson transformed \'y\'')
        
        plt.tight_layout()
        plt.show()
        
    # if(verbose):
    print('droped index: \n', outliers)
        
    df['SalePrice'] = y.copy()
    
    return df.copy()
    

### Combining training and test sets

In [5]:
def combine_tr_te(df_tr, df_te, verbose=False):
    
    all_d = pd.concat([d_tr.copy(), d_te.copy()])
    
    if(verbose):
        print('New data shape: ', all_d.shape)
        
    return all_d.copy()

### Imputting 'nan'

In [6]:

def impute_nan(all_data):
    
    # 34 columns with nan
    def fillWithNone(df):
        ''' nan in 'PoolQC' means 'no pool' 
            nan in 'MiscFeature' means 'no misc feature'
            nan in 'Alley' means 'no alley acces'
            nan in 'Fence' means 'no fence'
            nan in 'FireplaceQu' means 'no Fireplace'
            nan in 'GarageType', 'GarageFinish', 'GarageQual',
                'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1'
                'BsmtFinType1', 'MasVnrType', 'MSSubClass'
                'GarageCond' replaced with 'None' too

            recive a df
        '''

        df = df.copy()

        columns = ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 
                   'FireplaceQu', 'GarageType', 'GarageFinish', 
                   'GarageQual', 'GarageCond', 'BsmtQual',
                   'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
                   'BsmtFinType2', 'MasVnrType'
                  ]

        for col in columns:
            df[col] = df[col].fillna('None')

        return df

    def fillWithZero(df):
        ''' nan 

        '''

        df = df.copy()

        columns = ['GarageYrBlt', 'GarageArea', 'GarageCars',
                   'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
                   'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath',
                   'MasVnrArea'
                  ]

        for col in columns:
            df[col] = df[col].fillna(0)

        return df

    def fillWithMode(df):
        ''' fill missing values with mode, median
        '''
        df = df.copy()

        columns = ['Electrical', 'KitchenQual', 'Exterior1st',
                   'Exterior2nd', 'SaleType'
                  ]

        for col in columns:
            df[col] = df[col].fillna(df[col].mode()[0])

        # filling with median of each 'Neighborhood'
        df['LotFrontage'] = (
                         df.groupby('Neighborhood')['LotFrontage']
                         .transform(lambda x: x.fillna(x.median()))
                        )  

        # nan means Typical
        df['Functional'] = df['Functional'].fillna('Typ')

        return df


    def mszoning(df):
        ''' recives a DF this imputation takes place on test data only'''

        df = df.copy()

        idotrr = ( (df['Neighborhood'] == 'IDOTRR') & (df['MSZoning'].isna()) )
        mitchel = ( (df['Neighborhood'] == 'Mitchel') & (df['MSZoning'].isna()) )

        df.loc[ idotrr , 'MSZoning'] = 'RM'
        df.loc[ mitchel, 'MSZoning'] = 'RL'

    #     # to test this function out of here
    #     temp = mszoning(d_te)
    #     # lable index acces at [1916, 2217, 2251, 2905
    #     temp.loc[[1916, 2217, 2251, 2905], 'MSZoning']

        return df

    none_func = FunctionTransformer(fillWithNone, validate=False) 
    zero_func = FunctionTransformer(fillWithZero, validate=False) 
    mode_func = FunctionTransformer(fillWithMode, validate=False)
    mszo_func = FunctionTransformer(mszoning, validate=False)
    
    
    imputer = Pipeline([
                    ('withNone', none_func), 
                    ('withZero', zero_func), 
                    ('withMode', mode_func), 
                    ('mszoni', mszo_func)
                   ])
    
    result = imputer.fit_transform(all_data)
    
    return result.copy()

### Reduce Categories(Collapsing)

In [7]:
def reduce_categories(all_data):
    '''
    '''


    con1_others = ['RRAn', 'PosN', 'RRAe', 'PosA', 'RRNn', 'RRNe']
    roofS_others = ['Gambrel', 'Flat', 'Mansard', 'Shed']
    foun_others = ['Slab', 'Stone', 'Wood']
    gara_others = ['None', 'Basment', '2Types', 'CarPort']
    saleT_others = ['ConLD', 'CWD', 'ConLI', 'ConLw', 'Oth', 'Con']
    saleC_others = ['Family', 'Alloca', 'AdjLand']
    exte1_others = ['BrkComm', 'AsphShn', 'Stone', 'CBlock', 'ImStucc']
    exte2_others = ['BrkComm', 'AsphShn', 'Stone', 'CBlock', 'ImStucc', 'Other']
    lotC_others = ['FR2', 'FR3']

    all_data['Condition1'] = all_data['Condition1'].map(lambda x: 'Others' if x in con1_others else x)
    all_data['RoofStyle'] = all_data['RoofStyle'].map(lambda x: 'Others' if x in roofS_others else x)
    all_data['Foundation'] = all_data['Foundation'].map(lambda x: 'Others' if x in foun_others else x)
    all_data['GarageType'] = all_data['GarageType'].map(lambda x: 'Others' if x in gara_others else x)
    all_data['SaleType'] = all_data['SaleType'].map(lambda x: 'Others' if x in saleT_others else x)
    all_data['SaleCondition'] = all_data['SaleCondition'].map(lambda x: 'Others' if x in saleC_others else x)
    all_data['Exterior1st'] = all_data['Exterior1st'].map(lambda x: 'Others' if x in exte1_others else x)
    all_data['Exterior2nd'] = all_data['Exterior2nd'].map(lambda x: 'Others' if x in exte2_others else x)
    all_data['LotConfig'] = all_data['LotConfig'].map(lambda x: 'Others' if x in lotC_others else x)


    # Reducing to a BINARY CLASSES(just 2 clases)

    landC_others = ['HLS', 'Bnk', 'Low']
    cond2_others = ['Feedr', 'Artery', 'PosN', 'PosA', 'RRNn', 'RRAn', 'RRAe']
    roofM_others = ['Tar&Grv', 'WdShake', 'WdShngl', 'Metal', 'Membran', 'Roll', 'ClyTile']
    heati_others = ['GasW', 'Grav', 'Wall', 'OthW', 'Floor']
    elect_others = ['FuseA', 'FuseF', 'FuseP', 'Mix']
    miscF_others = ['Shed', 'Gar2', 'Othr', 'TenC']

    all_data['LandContour'] = all_data['LandContour'].map(lambda x: 'Others' if x in landC_others else x)
    all_data['Condition2'] = all_data['Condition2'].map(lambda x: 'Others' if x in cond2_others else x)
    all_data['RoofMatl'] = all_data['RoofMatl'].map(lambda x: 'Others' if x in roofM_others else x)
    all_data['Heating'] = all_data['Heating'].map(lambda x: 'Others' if x in heati_others else x)
    all_data['Electrical'] = all_data['Electrical'].map(lambda x: 'Others' if x in elect_others else x)

    all_data['MiscFeature'] = all_data['MiscFeature'].map(lambda x: 'Others' if x in miscF_others else x)
    
    
    return all_data.copy()

### Features Engineering

In [8]:

def features_engineering(all_data):
    # Before using d_tr set 
    # get the new d_tr set from all_data
    d_tr = all_data[all_data['Training']].copy()

    # Creating a new feature 'PeakMonths', 'Unfinished',
    # 'Splited', and TotalSF
    peak_moS = [5, 6, 7]
    unfi_hou = ['1.5Unf', '2.5Unf']
    spli_hou = ['SFoyer', 'SLvl']

    all_data['PeakMonths'] = all_data['MoSold'].map(lambda x: 'Peak' if x in peak_moS else 'Normal' )
    all_data['Finished']   = all_data['HouseStyle'].map(lambda x: 'no' if x in unfi_hou else 'yes') 
    all_data['Splited']    = all_data['HouseStyle'].map(lambda x: 'yes' if x in spli_hou else 'no')

    all_data['TotalSF']    = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']

    
    # ======== Clustering ==========
    # we will use the training data because if we uses all_data
    # it has 'nan' in 'SalePrice'. test data does not have 'SalePrice'
    # 'Neighborhood', 5, 'MSSubClass', 4
    nei_cluster = make_clusters(d_tr.copy(), 'Neighborhood', 'SalePrice', 5)
    mss_cluster = make_clusters(d_tr.copy(), 'MSSubClass', 'SalePrice', 4)

    # merging the clusters data frame with all_data DataFrame
    # we got a 'nan' cluster becouse 'MSSubClass' in test_DF
    # has a '150' class that is just in test
    # we preserved the index from all_d DF
    all_data = all_data.reset_index().merge(nei_cluster, how='left', on='Neighborhood').set_index('Id')
    # all_data.drop(columns='Neighborhood', inplace=True)

    all_data = all_data.reset_index().merge(mss_cluster, how='left', on='MSSubClass').set_index('Id')
    
    # dropping old columns
    # all_data.drop(columns=['Neighborhood', 'MSSubClass'], inplace=True)

    # all_data.reset_index(drop=True, inplace=True)
    
    # ========= Cycling Features ====
    all_data[ 'MoSold' + '_sin'] = np.sin( all_data['MoSold'] * (2.*np.pi/12) )
    all_data[ 'MoSold' + '_cos'] = np.cos( all_data['MoSold'] * (2.*np.pi/12) )
    
    # ========= Cleaning ============
    all_data.drop(columns=['Neighborhood', 'MSSubClass', 'MoSold'], inplace=True)
    
    # updating the ** new feature ** types
    all_data = all_data.astype( {'PeakMonths':str, 'Finished':str, 'Splited':str,
                                 'Neighborhood_Cluster': str, 'MSSubClass_Cluster': str} )#,
                                 # 'MoSold_sin': str, 'MoSold_cos': str } )
    
    return all_data.copy()

# # updating cat_to_1Hot
# cat_to_1Hot.update( {'PeakMonths':str, 'Finished':str, 'Splited':str,
#                        'Neighborhood_Cluster': str, 'MSSubClass_Cluster': str} )

### Global Variables
> ord_cat_mapping  
> cat_to_1Hot  
> ord_cat_DONE  

In [9]:
##### ****** All writed by me ****** #####
##########################################
def get_global_variables():
    ord_cat_mapping = [
        {
            'col': 'FireplaceQu',
            'mapping': {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
        },
        {
            'col': 'GarageQual',
            'mapping': {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
        },
        {
            'col': 'GarageCond',
            'mapping': {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
        },
        {
            'col': 'BsmtFinType1',
            'mapping': {'None': 0, 'Unf': 1, 'LwQ': 2, 'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6}
        },
        {
            'col': 'BsmtFinType2',
            'mapping': {'None': 0, 'Unf': 1, 'LwQ': 2, 'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6}
        },
        {
            'col': 'ExterQual',
            'mapping': {'Fa': 0, 'TA': 1, 'Gd': 2, 'Ex': 3}
        },
        {
            'col': 'ExterCond',
            'mapping': {'Po': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4}
        },
        {
            'col': 'BsmtQual',
            'mapping': {'None': 0 , 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4}
        },
        {
            'col': 'BsmtCond',
            'mapping': {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4}
        },
        {
            'col': 'PoolQC',
            'mapping': {'None': 0, 'Fa': 1, 'Gd': 2, 'Ex': 3}
        },
        {
            'col': 'HeatingQC',
            'mapping': {'Po': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4}
        },
        {
            'col': 'KitchenQual',
            'mapping': {'Fa': 0, 'TA': 1, 'Gd': 2, 'Ex': 3}
        },
        {
            'col': 'BsmtExposure',
            'mapping': {'None': 0, 'No': 1, 'Mn': 2, 'Av': 3, 'Gd': 4}
        },
        {
            'col': 'Functional',
            'mapping': {'Sev': 0, 'Maj2': 1, 'Maj1': 2, 'Mod': 3, 'Min2': 4, 'Min1': 5, 'Typ': 6}
        },
        {
            'col': 'GarageFinish',
            'mapping': {'None': 0, 'Unf': 1, 'RFn': 2, 'Fin': 3}
        },
        {
            'col': 'Fence',
            'mapping': {'None': 0, 'MnWw': 1, 'GdWo': 2, 'MnPrv': 3, 'GdPrv': 4}
        },
        {
            'col': 'CentralAir',
            'mapping': {'N': 0, 'Y': 1}
        },
        {
            'col': 'PavedDrive',
            'mapping': {'N': 0, 'P': 1, 'Y': 2}
        },
        {
            'col': 'Street',
            'mapping': {'Grvl': 0, 'Pave': 1}
        },
        {
            'col': 'Alley',
            'mapping': {'None': 0, 'Grvl': 1, 'Pave': 2}
        },
        {
            'col': 'LandSlope',
            'mapping': {'Gtl': 0, 'Mod': 1, 'Sev': 2}
        },
        {
            'col': 'LotShape',
            'mapping': {'Reg': 0, 'IR1': 1, 'IR2': 2, 'IR3': 3}
        },
        {
            'col': 'HouseStyle', 
            'mapping': {'SLvl': 0, 'SFoyer': 0, '1Story': 1, '1.5Fin': 2, 
                        '1.5Unf': 2, '2Story': 3, '2.5Unf': 4, '2.5Fin': 4}
        }
    ]

    # list of categorical columns(23)  
    # that we just encoded
    ord_cat_DONE = {'FireplaceQu': str, 'GarageQual': str,'GarageCond': str,'BsmtFinType1': str,
                     'BsmtFinType2': str,'ExterQual': str,'ExterCond': str,'BsmtQual': str,
                     'BsmtCond': str,'PoolQC': str,'HeatingQC': str,'KitchenQual': str,
                     'BsmtExposure': str,'Functional': str,'GarageFinish': str,'Fence': str, 
                     'CentralAir': str, 'PavedDrive': str,'Street': str,'Alley': str,
                     'LandSlope': str,'LotShape': str, 'HouseStyle': str}

    # to encode using OneHot (15 so far)
    cat_to_1Hot = {'Condition1': str, 'RoofStyle': str, 'Foundation': str, 'GarageType': str, 'SaleType': str, 
                   'SaleCondition': str, 'Exterior1st': str, 'Exterior2nd': str, 'LotConfig': str, 'LandContour': str, 
                   'Condition2': str, 'RoofMatl': str, 'Heating': str, 'Electrical': str, 'MiscFeature': str }

    # updating cat_to_1Hot with new created features
    cat_to_1Hot.update( {'PeakMonths':str, 'Finished':str, 'Splited':str,
                           'Neighborhood_Cluster': str, 'MSSubClass_Cluster': str} )

    # updating the types
    # update the list of ordinal with 2 columns name
    # that allredy are ordinal and encoded ('OverallQual', 'OverallCond')

    ord_cat_DONE.update({'OverallQual': str, 'OverallCond': str})#,
                         # 'MoSold_sin': str, 'MoSold_cos': str})
    # new_d = new_d.astype(ord_cat_DONE)

    # 5 more columns will concidere as categorical
    # we take off 'MoSold': str, 
    cat_to_1Hot.update({'YrSold': str, 'BldgType':str, 
                        'MSZoning': str, 'MasVnrType': str})

    # new_d = new_d.astype(cat_to_1Hot)
    
    # updating the data type
    # df = df.astype(cat_to_1Hot)
    # df = df.astype(ord_cat_DONE)
    
    return (ord_cat_mapping, ord_cat_DONE, cat_to_1Hot)

### Ordinal and OneHot encoding

In [10]:
def ordinal_1hot_encode(all_data, ord_mapping, oneHot_col):
    
    ## OrdinalEncoder
    oe = OrdinalEncoder(mapping=ord_mapping).fit(all_data)
    
    all_data = oe.transform(all_data)
    
    
    ## OneHot encoding
    oh = OneHotEncoder(cols=oneHot_col).fit(all_data)
    
    all_data = oh.transform(all_data.copy())
    
    
    
    return all_data.copy()

### Feature Selection

In [11]:
def feature_selection(X, y, verbose=False, sorted_features=False):
    
    mutual_info = mutual_info_regression(X=X, y=y)
    
    mu_info_df = pd.DataFrame(list(zip( X.columns, mutual_info )), columns=['Features', 'Mutual_info'])
    
    feature_to_dop = mu_info_df[ mu_info_df['Mutual_info'] == 0]
    feature_to_dop = list( feature_to_dop['Features'] )
    
    if(verbose):
        #Sorting
        # mu_info_df.sort_values('Mutual_info', ascending=False, inplace=True)
        print('{} Features with Zero(0) mutual info: \n'.format(len(feature_to_dop)))
        feature_to_dop = sorted(feature_to_dop)
        print(feature_to_dop)
        
    if( (verbose == False) and sorted_features):
        feature_to_dop = sorted(feature_to_dop)
        
    
    return (feature_to_dop, mu_info_df.sort_values('Mutual_info', ascending=False).copy() )

### Fix skewed columns

In [12]:
def fix_skew_cols(df, cols, treshold=0.7):
    '''
    this function scals and fix the skewed columns
    by applying RobustScaler() and 'yeo-johnson' transform
    '''
    
    skewed = all_d[cols].skew()
    
    skewed = skewed[skewed >= 0.7]
    skewed = skewed.index
    
    print('{} features with skewe >= {}'.format( len(skewed), treshold ) )
    
    fix_data_skew = Pipeline([('scaler', RobustScaler()), 
                          ('yeo', PowerTransformer(method='yeo-johnson'))])
    
    df[skewed] = fix_data_skew.fit_transform(df[skewed].copy())
    
    return df.copy()

### Polynomial Features

In [13]:
def poly_features(df, top_cols):
    '''
    function to create the interaction feature of the top_cols
    
    '''
    
    top_cols_set = df[top_cols].copy()
    # top_cols_set
    
    poly_features = PolynomialFeatures(2, interaction_only=True, include_bias=False)
    
    index_backup = top_cols_set.index.copy()
    
    poly_set = pd.DataFrame( poly_features.fit_transform(top_cols_set.copy()), columns=poly_features.get_feature_names_out(top_cols) )
    poly_set.set_index(index_backup, inplace=True)
    # poly_set
    
    poly_set.drop(columns=top_cols, inplace=True)
    # poly_set
    
    result = df.merge(poly_set, left_index=True, right_index=True)
    
    return result.copy()
    

# Driver

In [14]:
# get the data
d_tr, d_te = data(verbose=True)

d_tr shape: (1460, 80)
d_te shape: (1459, 79)


In [15]:
# taking cara of some outliers
# and making 'y' more gausian like
d_tr = analyze_y(d_tr.copy())

droped index: 
 Int64Index([524, 1299], dtype='int64', name='Id')


In [16]:
# after take care of some outlier
# combine the training and the test
all_d = combine_tr_te(d_tr.copy(), d_te.copy(), verbose=True)
all_d.shape

New data shape:  (2917, 80)


(2917, 80)

### Imputing

In [17]:
# imputing nan
all_d = impute_nan(all_d.copy())

In [18]:
all_d.columns[all_d.isna().any()].to_list()

['SalePrice']

### Reducing Cardinality

In [19]:
# reducing the number of categories in some columns
all_d = reduce_categories(all_d.copy())
# all_d['Foundation'].value_counts()

### Feature Engineering

In [20]:
# Creating a new feature 'PeakMonths', 'Unfinished',
# 'Splited', and TotalSF, and Clusters 
# transforming cyclical feature 'MoSold'
all_d = features_engineering(all_d.copy())

### Identifying  
> Categorical and Number Columns

In [21]:
# getting the global variables
ord_cat_mapping, ord_cat_DONE, cat_to_1Hot = get_global_variables()

In [22]:
# updating the data type
all_d = all_d.astype(cat_to_1Hot)
all_d = all_d.astype(ord_cat_DONE)

In [23]:
# after updating all the categorical columns
# we can now identify the number columns
numbers_col = all_d.select_dtypes('number')
numbers_col = numbers_col.columns.to_list()

# removing the transformed cyclical feature
# that already are float and they were identifyed
# as numerical features and we dont want to
# touch them
numbers_col.remove('MoSold_sin')
numbers_col.remove('MoSold_cos')
# MoSold_sin, MoSold_cos

In [ ]:
analyzed_col = numbers_col + list(cat_to_1Hot.keys()) + list(ord_cat_DONE.keys())
set( all_d.columns.to_list() ) - set( analyzed_col )

In [ ]:
set( analyzed_col ) - set( all_d.columns.to_list() )

In [ ]:
print('analyzed_col len:', len(analyzed_col))
print('all_d.columns len', len(all_d.columns.to_list()))

### Fixing skewnnes on Number Columns

In [24]:
# the DF returned will have numbers_col Scalered and 
# PowerTransformed
all_d = fix_skew_cols(all_d.copy(), numbers_col) # this function keeps the index

22 features with skewe >= 0.7


### Doing Ordinal and OneHot Encoding

In [25]:
# encoding ordinal and onehot columns( keeps Id index)
all_d = ordinal_1hot_encode(all_d.copy(), ord_cat_mapping, cat_to_1Hot)

In [ ]:
all_d

### Feature Selection

In [26]:
# getting the training set to work with
d_tr = all_d[ all_d['Training'] ].copy()

In [29]:
# we will get the features that has 0 mutual information with 'y'
# and a DF with **ALL the features** with their mutual info value
# from it we can get the top ones, to do polynomial interaction
to_drop, mutual_info_df = feature_selection( d_tr.drop(columns='SalePrice').copy(), 
                                            d_tr['SalePrice'].copy(), verbose=True, sorted_features=True)

23 Features with Zero(0) mutual info: 

['BldgType_2', 'Condition1_3', 'Condition2_2', 'Exterior1st_10', 'Exterior1st_11', 'Exterior1st_5', 'Exterior1st_8', 'Exterior2nd_7', 'Exterior2nd_8', 'Exterior2nd_9', 'LotConfig_2', 'MSZoning_5', 'MiscFeature_1', 'MiscFeature_2', 'MiscVal', 'MoSold_sin', 'Neighborhood_Cluster_2', 'PoolArea', 'PoolQC', 'RoofMatl_1', 'RoofMatl_2', 'RoofStyle_3', 'YrSold_3']


### Polynomial and Interaction Features

In [30]:
# from mutual_info_df get the top 20 features
# that has the hight mutual information
# becouse it is ordered we can take the top 20
top_20_cols = mutual_info_df.head(20)['Features'].to_list()
print( top_20_cols )

['TotalSF', 'OverallQual', 'GrLivArea', 'GarageArea', 'GarageCars', 'TotalBsmtSF', 'YearBuilt', 'BsmtQual', 'ExterQual', 'KitchenQual', '1stFlrSF', 'GarageFinish', 'FullBath', 'GarageYrBlt', 'YearRemodAdd', 'TotRmsAbvGrd', 'FireplaceQu', '2ndFlrSF', 'LotFrontage', 'LotArea']


In [31]:
all_d.shape

(2917, 158)

In [32]:
# we will get the interaction factor por the top 20 
all_d = poly_features(all_d.copy(), top_20_cols)

In [33]:
all_d.shape

(2917, 348)

> Before modeling drop the features with 0 mutual info  
> beacous the list has 'Training'

In [ ]:
# we keep 'Training' column to identify training set in all_d
to_drop.remove('Training')

In [35]:
# aditional features that we dont 
# want to remove
to_drop.remove('MoSold_sin')

In [36]:
all_d.shape, len(to_drop)

((2917, 348), 22)

In [37]:
# 'Training' is in to_drop list
all_d.drop(columns=to_drop, inplace=True)

In [38]:
all_d.shape

(2917, 326)

> Getting **the Training, Test set** and 'y' back

In [148]:
tr = all_d[ all_d['Training'] ].copy()
te = all_d[  ~all_d['Training'] ].copy()

In [149]:
y = tr[ 'SalePrice' ].copy()

tr.drop(columns='SalePrice', inplace=True)
te.drop(columns='SalePrice', inplace=True)

### Outliers detection

> Using **IsolationForest**

In [150]:
iso_forest = IsolationForest()

In [151]:
y_hat = iso_forest.fit_predict(tr)

In [152]:
inliers_mask = y_hat != -1

In [153]:
print( 'There are {} outliers'.format(np.count_nonzero(y_hat == -1)))

There are 108 outliers


In [160]:
# reshape to print the indexes in a block
print(np.argwhere(y_hat == -1).reshape(1,-1))

[[  11   35   53   58   75   87   88  112  113  118  161  167  178  185
   196  197  224  231  238  249  261  278  304  309  313  320  324  343
   349  350  362  378  384  389  417  430  434  440  473  477  489  496
   515  526  528  532  539  580  582  590  609  613  617  634  635  641
   643  648  663  690  701  704  723  744  746  768  773  783  797  802
   824  828  897  913  931  933  986  993  999 1023 1035 1043 1045 1057
  1099 1141 1168 1172 1180 1181 1227 1229 1242 1266 1267 1303 1335 1348
  1351 1357 1362 1371 1384 1386 1403 1435 1440 1447]]


> Removing outliers

In [154]:
tr.shape

(1458, 325)

In [155]:
tr = tr.iloc[inliers_mask, : ]
y  = y[inliers_mask]

In [156]:
tr.shape

(1350, 325)

> using **residuals**

In [161]:
lr = LinearRegression()

In [162]:
lr.fit(tr, y)

LinearRegression()

In [163]:
y_hat = lr.predict(tr)

In [164]:
residuals = y - y_hat
standard_residuals = abs(  ( residuals - residuals.mean() ) / residuals.std() )

In [165]:
inliers_mask = standard_residuals <= 3

In [166]:
print( 'there are {} new outliers'.format( np.count_nonzero( (standard_residuals >= 3) ) ))

there are 15 new outliers


In [167]:
# index dropped
inliers_mask[ inliers_mask == False].index

Int64Index([309, 411, 463, 561, 629, 633, 689, 715, 773, 969, 971, 1023, 1325,
            1433, 1454],
           dtype='int64', name='Id')

> **removing** new outliers

In [168]:
tr.shape

(1350, 325)

In [169]:
tr = tr.loc[ inliers_mask ]

In [170]:
y = y[ inliers_mask ]

In [171]:
tr.shape, y.shape

((1335, 325), (1335,))

### Modeling

In [147]:
y.info()

<class 'pandas.core.series.Series'>
Int64Index: 1458 entries, 1 to 1460
Series name: SalePrice
Non-Null Count  Dtype  
--------------  -----  
1458 non-null   float64
dtypes: float64(1)
memory usage: 22.8 KB


In [135]:
tr.loc[308:311]

,MSZoning_1,MSZoning_2,MSZoning_3,MSZoning_4,LotFrontage,LotArea,Street,Alley,LotShape,LandContour_1,...,TotRmsAbvGrd FireplaceQu,TotRmsAbvGrd 2ndFlrSF,TotRmsAbvGrd LotFrontage,TotRmsAbvGrd LotArea,FireplaceQu 2ndFlrSF,FireplaceQu LotFrontage,FireplaceQu LotArea,2ndFlrSF LotFrontage,2ndFlrSF LotArea,LotFrontage LotArea
Id,,,,,,,,,,,,,,,,,,,,,
308,0,1,0,0,-0.401931,-0.279561,1,1,1,1,...,-0.000000,-0.107180,0.073067,0.050821,0.000000,-0.000000,-0.000000,-0.236971,-0.164824,0.112364
310,1,0,0,0,0.985521,0.687203,1,0,1,1,...,5.072473,-0.862727,0.999805,0.697164,-4.252004,4.927604,3.436017,-0.838088,-0.584398,0.677253
311,1,0,0,0,-0.195436,-0.349449,1,0,1,1,...,-0.545369,-0.221250,0.035528,0.063526,3.651194,-0.586309,-1.048348,-0.237859,-0.425303,0.068295


In [137]:
d_tr.loc[405:415]

,MSZoning_1,MSZoning_2,MSZoning_3,MSZoning_4,LotFrontage,LotArea,Street,Alley,LotShape,LandContour_1,...,TotRmsAbvGrd FireplaceQu,TotRmsAbvGrd 2ndFlrSF,TotRmsAbvGrd LotFrontage,TotRmsAbvGrd LotArea,FireplaceQu 2ndFlrSF,FireplaceQu LotFrontage,FireplaceQu LotArea,2ndFlrSF LotFrontage,2ndFlrSF LotArea,LotFrontage LotArea
Id,,,,,,,,,,,,,,,,,,,,,
405,1,0,0,0,-0.195436,0.324390,1,0,1,1,...,1.381912,0.543793,-0.090025,0.149426,3.541571,-0.586309,0.973169,-0.230717,0.382950,-0.063397
406,1,0,0,0,0.196351,0.246339,1,0,1,1,...,3.043484,-0.862727,0.199197,0.249910,-2.551202,0.589052,0.739017,-0.166977,-0.209487,0.048369
407,1,0,0,0,-0.887806,0.347829,1,0,0,1,...,-0.000000,0.154594,0.161394,-0.063232,-0.000000,-0.000000,0.000000,0.754991,-0.295794,-0.308804
408,1,0,0,0,-0.246457,1.135446,1,0,0,1,...,0.000000,1.267672,-0.250029,1.151904,0.000000,-0.000000,0.000000,-0.307962,1.418808,-0.279838
412,1,0,0,0,1.388730,2.699165,1,0,0,0,...,-0.000000,0.813996,-1.329279,-2.583616,-0.000000,0.000000,0.000000,-1.180977,-2.295372,3.748412
413,0,0,0,1,0.219811,-1.524430,1,0,2,1,...,1.842550,-0.391726,0.101253,-0.702210,-3.401603,0.879242,-6.097722,-0.186927,1.296377,-0.335086
414,0,1,0,0,-0.614475,0.005146,1,1,0,1,...,-3.828762,0.813996,0.588170,-0.004926,-3.401603,-2.457900,0.020584,0.522550,-0.004376,-0.003162
415,1,0,0,0,-0.454524,0.490666,1,0,2,1,...,4.531836,2.091321,-0.686610,0.741207,4.153260,-1.363573,1.471999,-0.629253,0.679288,-0.223020


In [143]:
d_tr.loc[965:975, 'Training']

Id
965    True
966    True
967    True
968    True
970    True
972    True
973    True
974    True
975    True
Name: Training, dtype: bool

In [142]:
y.loc[965:975]

,SalePrice
Id,
965,0.685044
966,0.237180
967,-0.056528
968,-0.533016
970,-0.429583
972,0.150980
973,-1.381308
974,0.280746
975,0.066367
